This note book is to do experiment on building main contracts based on open interest

# Check day bar data for all the symbol roots

In [1]:
import glob
import os

In [2]:
data_folder_path = r"D:\data\iqfeed\future\future_data_day"

In [3]:
data_path_list = glob.glob(os.path.join(data_folder_path, "*.csv"))

In [4]:
iqfeed_symbol_list = [os.path.basename(path)[:-8] for path in data_path_list if "#"not in path]

In [5]:
iqfeed_symbol_root_list = [symbol[:-1] for symbol in iqfeed_symbol_list]

In [6]:
iqfeed_symbol_root_list = list(set(iqfeed_symbol_root_list))

In [7]:
iqfeed_symbol_root_list

['@CC',
 '@SB',
 '@ES',
 '@YC',
 '@QM',
 'IHO',
 '@DA',
 '@QG',
 '@OJ',
 '@QO',
 '@MES',
 'QGC',
 'QMGC',
 '@KC',
 '@MYM',
 '@NQ',
 '@RTY',
 '@M2K',
 '@YM',
 '@QC',
 '@MNQ',
 '@CT']

# check missing data for day bar

In [8]:
import pandas as pd
symbol_meta_frame = pd.read_csv(r"E:\project\trade_engine\iqfeed_data_processor\futures_to_import.csv")

In [9]:
all_symbols_list = symbol_meta_frame.loc[:, "symbol_roots"].tolist()

In [10]:
[a for a in all_symbols_list if a not in iqfeed_symbol_root_list]

['@CL']

In [11]:
[a for a in iqfeed_symbol_root_list if a not in all_symbols_list]

[]

In [190]:
for symbol_root in iqfeed_symbol_root_list:
    print(symbol_root)
    # According to symbol meta frame, figure out the ib symbol root
    ib_symbol_root = symbol_meta_frame[symbol_meta_frame["symbol_roots"] == symbol_root].loc[:,"symbol_ib_roots"].iloc[0]
    symbols_path_list = [path for path in data_path_list if (symbol_root in path and "#" not in path)]
    # combine all of the sub symbols frames into one frame
    one_symbol_combined_frame = pd.DataFrame()
    for path in symbols_path_list:
        frame = pd.read_csv(path)

        # According to iqfeed day bar data path string, figure out the iqfeed symbol root
        symbol = os.path.basename(path)[:-6]
        symbol_sub_code = os.path.basename(path)[-9:-6]
        symbol_root = os.path.basename(path)[:-9]

        # find the corresponding ib_symbol. ib_symbol has the same sub code compared to iqfeed symbol
        ib_symbol = "".join([ib_symbol_root, symbol_sub_code])
        frame.columns = ["datetime", "open", "high", "low", "close", "volume", "open_interest"]
        datetime_format = '%Y%m%d %H:%M:%S'
        frame['datetime'] = pd.to_datetime(frame['datetime'], format=datetime_format)
        float_columns = ['open', 'high', 'low', 'close', 'volume', 'open_interest']
        for col in float_columns:
            frame.loc[:, col] = frame[col].astype('float')
        frame.loc[:, "symbol"] = ib_symbol
        # We should use the open interest of last day to determine the main contract
        frame.loc[:, "last_open_interest"] = frame.loc[:, "open_interest"].shift(1).fillna(0.0) 
        one_symbol_combined_frame = pd.concat([one_symbol_combined_frame, frame])

    one_symbol_combined_frame = one_symbol_combined_frame.reset_index(drop=True)
    one_symbol_combined_frame = one_symbol_combined_frame.sort_values(by=["datetime", "last_open_interest", "open_interest"], axis=0)
    one_symbol_combined_frame = one_symbol_combined_frame.drop_duplicates(subset="datetime", keep="last")

    one_symbol_combined_frame.to_csv(ib_symbol_root + ".csv", index=False)

@CT
@KC
@YC
@MES
@QM
@NQ
@QO
@ES
@QG
@M2K
QGC
@DA
@CC
@QC
@MNQ
@OJ
IHO
@RTY
@MYM
@SB
@YM
QMGC


In [12]:
symbol_root = "@SB"
print(symbol_root)
# According to symbol meta frame, figure out the ib symbol root
ib_symbol_root = symbol_meta_frame[symbol_meta_frame["symbol_roots"] == symbol_root].loc[:,"symbol_ib_roots"].iloc[0]
symbols_path_list = [path for path in data_path_list if (symbol_root in path and "#" not in path)]
# combine all of the sub symbols frames into one frame
one_symbol_combined_frame = pd.DataFrame()
for path in symbols_path_list:
    frame = pd.read_csv(path)

    # According to iqfeed day bar data path string, figure out the iqfeed symbol root
    symbol = os.path.basename(path)[:-6]
    symbol_sub_code = os.path.basename(path)[-9:-6]
    symbol_root = os.path.basename(path)[:-9]

    # find the corresponding ib_symbol. ib_symbol has the same sub code compared to iqfeed symbol
    ib_symbol = "".join([ib_symbol_root, symbol_sub_code])
    frame.columns = ["datetime", "open", "high", "low", "close", "volume", "open_interest"]
    datetime_format = '%Y%m%d %H:%M:%S'
    frame['datetime'] = pd.to_datetime(frame['datetime'], format=datetime_format)
    float_columns = ['open', 'high', 'low', 'close', 'volume', 'open_interest']
    for col in float_columns:
        frame.loc[:, col] = frame[col].astype('float')
    frame.loc[:, "symbol"] = ib_symbol
    # We should use the open interest of last day to determine the main contract
    frame.loc[:, "last_open_interest"] = frame.loc[:, "open_interest"].shift(1).fillna(0.0) 
    one_symbol_combined_frame = pd.concat([one_symbol_combined_frame, frame])

one_symbol_combined_frame = one_symbol_combined_frame.reset_index(drop=True)
one_symbol_combined_frame = one_symbol_combined_frame.sort_values(by=["datetime", "last_open_interest", "open_interest"], axis=0)
one_symbol_combined_frame = one_symbol_combined_frame.drop_duplicates(subset="datetime", keep="last")


@SB


In [30]:
one_symbol_combined_frame.head()

,datetime,open,high,low,close,volume,open_interest,symbol,last_open_interest
828,2006-01-03,14.65,14.67,14.00,14.18,40010.0,279561.0,SBH06,0.0
829,2006-01-04,14.35,14.90,14.35,14.87,35040.0,280845.0,SBH06,279561.0
830,2006-01-05,14.95,15.17,14.70,14.85,29564.0,276754.0,SBH06,280845.0
831,2006-01-06,14.90,15.10,14.82,14.88,28222.0,271202.0,SBH06,276754.0
832,2006-01-09,14.80,14.89,14.55,14.60,26003.0,269602.0,SBH06,271202.0
